In [7]:
import linecache
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import datasets
from transformers import AutoTokenizer
import json


class LazyTextDataset(Dataset):
    def __init__(self, filename):
        self._filename = filename
        self._total_data = 0
        self._tokenizer = AutoTokenizer.from_pretrained("xlmr-roberta-base")
        with open(filename, "r") as f:
            self._total_data = len(f.readlines()) - 1

    def __getitem__(self, idx):
        line = linecache.getline(self._filename, idx + 1)
        data = json.loads(line)
        data = self._tokenizer([data],return_tensors='pt')
        return data
        
    def __len__(self):
        return self._total_data
    
class datasetsWrapper(Dataset):
    def __init__(self, filename):
        self._filename = filename
        self._dataset = datasets.load_dataset("json",data_files=[filename])['train']
        self._total_data = len(self._dataset)
        self._tokenizer = AutoTokenizer.from_pretrained("xlmr-roberta-base")
        
    def preprocessing(self):
        self._dataset = self._dataset.map(
            preprocess,
            with
        )

    def __getitem__(self, idx):
        data = self._dataset[idx]
        return data
        
    def __len__(self):
        return self._total_data
    

filename = '/home/chenzhongzhi/czz/datasets/la13m_para5m_multilingual/laion2b_multi18lg/part-00054-66bc9e17-4cc9-4c76-aaed-af85e99e94d2-c000.snappy.json'
lcache_dataset = LazyTextDataset(filename=filename)
dsets_dataset = datasetsWrapper(filename=filename)



Using custom data configuration default-8f9316c872fa533b
Found cached dataset json (/home/chenzhongzhi/.cache/huggingface/datasets/json/default-8f9316c872fa533b/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
from time import time
from tqdm import tqdm

batch_size = 512
lcache_dataloader = DataLoader(lcache_dataset,batch_size=batch_size)
dsets_dataloader = DataLoader(dsets_dataset,batch_size=batch_size)

beg = time()
for i in tqdm(lcache_dataloader):
    data = i 
dur = beg - time()


beg = time()
for i in tqdm(dsets_dataloader):
    data = i 
dur_ = beg - time()

print(dur,dur_)

 52%|█████▏    | 11485/22255 [07:57<07:25, 24.16it/s]